In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.preprocessing import StandardScaler
from transformers import pipeline

# Define the SentimentModel for your custom model (for multi-class classification)
class SentimentModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SentimentModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Apply ReLU activation
        x = self.fc2(x)  # Final output layer
        return x  # Logits (raw scores)

# Define your sentiment prediction function
def get_sentiment_score(comment):
    # Using HuggingFace's pipeline for sentiment analysis
    sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    sentiment_result = sentiment_pipeline(comment)[0]
    
    # Mapping the sentiment to a numeric score
    if sentiment_result['label'] == 'POSITIVE':
        sentiment_score = 1
    elif sentiment_result['label'] == 'NEGATIVE':
        sentiment_score = -1
    else:
        sentiment_score = 0  # For neutral
    sentiment_confidence = sentiment_result['score']
    
    return sentiment_score, sentiment_confidence

# Define the prediction function with model integration
def predict_sentiment_with_model(comment):
    # Get sentiment score from HuggingFace sentiment model
    sentiment_score, sentiment_confidence = get_sentiment_score(comment)

    # Simulate QoS data for the new comment (4 features)
    qos_test_data = np.random.rand(1, 4)

    # Add sentiment score to QoS data (5 features)
    qos_test_data_with_sentiment = np.hstack([qos_test_data, np.array([[sentiment_score]])])

    # Scale the features using the same scaler as the training data
    qos_test_data_with_sentiment = scaler.transform(qos_test_data_with_sentiment)

    # Convert to tensor and predict with the custom model
    input_tensor = torch.tensor(qos_test_data_with_sentiment, dtype=torch.float32)
    with torch.no_grad():
        output = model(input_tensor)

        # Apply softmax to get probabilities
        probabilities = F.softmax(output, dim=1)

        # Get the predicted class index (0: Negative, 1: Neutral, 2: Positive)
        predicted_class = torch.argmax(probabilities, dim=1).item()

    sentiment_labels = ['Negative', 'Neutral', 'Positive']
    predicted_sentiment = sentiment_labels[predicted_class]

    return predicted_sentiment, sentiment_confidence

# Initialize model and scaler (use your trained model and scaler)
input_size = 5  # 4 features from QoS + 1 sentiment score
hidden_size = 64
output_size = 3  # For 3 classes: Negative, Neutral, Positive
model = SentimentModel(input_size, hidden_size, output_size)

# Example training data and scaler (you would replace this with your actual training code)
X_train = np.random.rand(100, 4)  # 100 samples, 4 features each
y_train = np.random.randint(0, 3, size=(100,))  # Random labels (0, 1, 2 for Negative, Neutral, Positive)

# Add the sentiment score to the training data (this is a simulated example)
sentiment_scores = np.random.randint(-1, 2, size=(100, 1))  # Random sentiment score (-1, 0, 1)
X_train_with_sentiment = np.hstack([X_train, sentiment_scores])

# Scale the training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_with_sentiment)

# Training loop (simplified)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Convert data to tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

# Train the model
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

# Example usage for prediction
new_comment = "This movie was absolutely fantastic!"
predicted_sentiment, score = predict_sentiment_with_model(new_comment)
print(f"Predicted Sentiment: {predicted_sentiment} with confidence score: {score:.2f}")


Device set to use cpu


Predicted Sentiment: Positive with confidence score: 1.00
